In [1]:
import os
from selenium import webdriver
from seleniumwire import webdriver as swd
from time import sleep
import chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os

dir_path = os.getcwd()
profile = os.path.join(dir_path, "profile", "wpp")

options = webdriver.ChromeOptions()
#options.add_argument(r"user-data-dir={}".format(profile))

path = chromedriver_autoinstaller.install()
navegador = swd.Chrome(options=options)
navegador2 = swd.Chrome(options=options)
navegador3 = swd.Chrome(options=options)
sleep(2)

navegador.get('https://oauth.deriv.com/oauth2/authorize?app_id=16929&l=EN&signup_device=desktop&date_first_contact=2023-07-08&brand=deriv')
def login_deriv(driver):
    while len(driver.find_elements(By.XPATH, "//input[@type='email']")) < 1: pass
    sleep(.5)
    driver.find_element(By.XPATH, "//input[@type='email']").click()
    sleep(.3)
    driver.find_element(By.XPATH, "//input[@type='email']").send_keys(os.getenv('deriv_user'))
    sleep(.5)
    driver.find_element(By.XPATH, "//input[@type='password']").click()
    sleep(.3)
    driver.find_element(By.XPATH, "//input[@type='password']").send_keys(os.getenv('deriv_pass'))
    sleep(.8)
    driver.find_element(By.XPATH, "//button[@type='submit']").send_keys(Keys.ENTER)
    sleep(1)

login_deriv(navegador)
login_deriv(navegador2)
login_deriv(navegador3)

In [7]:
import json

for req in navegador.requests:
    if 'websockets' in req.url:
        sk = req

print(len(sk.ws_messages))
sk

123


Request(method='GET', url='wss://blue.binaryws.com/websockets/v3?app_id=16929&l=EN&brand=deriv', headers=[('Host', 'blue.binaryws.com'), ('Connection', 'Upgrade'), ('Pragma', 'no-cache'), ('Cache-Control', 'no-cache'), ('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'), ('Upgrade', 'websocket'), ('Origin', 'https://app.deriv.com'), ('Sec-WebSocket-Version', '13'), ('Accept-Encoding', 'gzip, deflate, br'), ('Accept-Language', 'en-US,en;q=0.9'), ('Sec-WebSocket-Key', 'kgSM9wEKEpGtVgMzUf6ZLw=='), ('Sec-WebSocket-Extensions', 'permessage-deflate; client_max_window_bits')], body=b'')

In [11]:
def contabilizador3(lista_valores):
    valores_unicos = range(10)
    porcentagens = [lista_valores.count(digit) / len(lista_valores) for digit in valores_unicos]
    
    menor_porcentagem = min(porcentagens)
    menor_digit = [digit for digit, porc in zip(valores_unicos, porcentagens) if porc == menor_porcentagem]
    
    print(f"Dígito(s) com menor porcentagem: {menor_digit}")
    #print(f"Lista atual: {lista_valores}")
    if len(menor_digit) == 1:
        return (True, menor_digit[0])
    else:
        return (False, False)
    
last_track = -1
last_tick = -1
sequencia = False
sw_nav = 0
while True:
    try:
        if sk.ws_messages[-1:] != last_track:
            lista_last_d_tick = []
            last_track = sk.ws_messages[-1:]
            for ws in sk.ws_messages[-200:]:
                try:
                    valor = json.loads(str(ws))['tick']['quote']
                    last_tick = int(str(valor)[-1]) if len(str(valor).split('.')[-1]) == 2 else 0
                    lista_last_d_tick.append(last_tick)
                except: pass
            action_1, last_1 = contabilizador3(lista_last_d_tick)
            action_2, last_2 = contabilizador3(lista_last_d_tick[-200:])
            action_3, last_3 = contabilizador3(lista_last_d_tick[-200:])
            action_4, last_4 = contabilizador3(lista_last_d_tick[-50:])
            if len(set([action_1,action_2,action_3,action_4])) == 1 and action_1 == True and len(set([last_1,last_2,last_3,last_4])) == 1 and int(navegador.find_element(By.XPATH, f'//span[@class="number-selector__selection number-selector__selection--selected"]').text) != last_1:
                #if action_1 and lista_last_d_tick[-1:].count(last_1) == 1:
                navegador.find_element(By.XPATH, f'//span[contains(@class,"number") and contains(text(), {last_1})]').click()
                navegador2.find_element(By.XPATH, f'//span[contains(@class,"number") and contains(text(), {last_1})]').click()
                navegador3.find_element(By.XPATH, f'//span[contains(@class,"number") and contains(text(), {last_1})]').click()
            elif len(set([action_1,action_2,action_3,action_4])) == 1 and action_1 == True and len(set([last_1,last_2,last_3,last_4])) == 1 and int(navegador.find_element(By.XPATH, f'//span[@class="number-selector__selection number-selector__selection--selected"]').text) == last_1:
                #elif action_1 == True and int(navegador.find_element(By.XPATH, f'//span[@class="number-selector__selection number-selector__selection--selected"]').text) == last_1 and action_1 and lista_last_d_tick[-2:].count(last_1) == 0:
                while True:
                    try:
                        if sw_nav == 0:
                            navegador.find_element(By.XPATH, f'//*[@id="dt_purchase_digitmatch_button"]/div[1]').click()
                            sw_nav += 1
                            print(f'====> Compra para igual: {last_1}')
                            sleep(0.4)
                            break
                        elif sw_nav == 1:
                            navegador2.find_element(By.XPATH, f'//*[@id="dt_purchase_digitmatch_button"]/div[1]').click()
                            sw_nav += 1
                            print(f'====> Compra para igual: {last_1}')
                            sleep(0.4)
                            break
                        elif sw_nav == 2:
                            navegador2.find_element(By.XPATH, f'//*[@id="dt_purchase_digitmatch_button"]/div[1]').click()
                            sw_nav == 0
                            print(f'====> Compra para igual: {last_1}')
                            sleep(0.4)
                            break
                    except: print('Erro em compra')
                #else:
                    #navegador.find_element(By.XPATH, f'//span[contains(@class,"number") and contains(text(), {last_1})]').click()
            else: pass
            sleep(.5)
    except IndexError: print('Erro')

Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [0]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [0]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [0]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [0, 3]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [0, 3, 6]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [0, 3, 6]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor por

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@class="number-selector__selection number-selector__selection--selected"]"}
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00606E73+48323]
	(No symbol) [0x00599661]
	(No symbol) [0x004A5308]
	(No symbol) [0x004D0B45]
	(No symbol) [0x004D0CDB]
	(No symbol) [0x004FE3D2]
	(No symbol) [0x004EA924]
	(No symbol) [0x004FCAC2]
	(No symbol) [0x004EA6D6]
	(No symbol) [0x004C847C]
	(No symbol) [0x004C957D]
	GetHandleVerifier [0x0086FD5D+2575277]
	GetHandleVerifier [0x008AF86E+2836158]
	GetHandleVerifier [0x008A96DC+2811180]
	GetHandleVerifier [0x006941B0+626688]
	(No symbol) [0x005A314C]
	(No symbol) [0x0059F4B8]
	(No symbol) [0x0059F59B]
	(No symbol) [0x005921B7]
	BaseThreadInitThunk [0x76317D59+25]
	Rtl

In [12]:
def contabilizador3(lista_valores):
    valores_unicos = range(10)
    porcentagens = [lista_valores.count(digit) / len(lista_valores) for digit in valores_unicos]
    
    menor_porcentagem = min(porcentagens)
    menor_digit = [digit for digit, porc in zip(valores_unicos, porcentagens) if porc == menor_porcentagem]
    
    print(f"Dígito(s) com menor porcentagem: {menor_digit}")
    #print(f"Lista atual: {lista_valores}")
    if len(menor_digit) == 1:
        return (True, menor_digit[0])
    else:
        return (False, False)
    
last_track = -1
last_tick = -1
sequencia = False
while True:
    try:
        if sk.ws_messages[-1:] != last_track:
            lista_last_d_tick = []
            last_track = sk.ws_messages[-1:]
            for ws in sk.ws_messages[-200:]:
                try:
                    valor = json.loads(str(ws))['tick']['quote']
                    last_tick = int(str(valor)[-1]) if len(str(valor).split('.')[-1]) == 2 else 0
                    lista_last_d_tick.append(last_tick)
                except: pass
            action, last = contabilizador3(lista_last_d_tick)
            if action and lista_last_d_tick[-1:].count(last) == 1:
                navegador.find_element(By.XPATH, f'//span[contains(@class,"number") and contains(text(), {last})]').click()
            elif action == True and int(navegador.find_element(By.XPATH, f'//span[@class="number-selector__selection number-selector__selection--selected"]').text) == last and action and lista_last_d_tick[-2:].count(last) == 0:
                while True:
                    try: 
                        navegador.find_element(By.XPATH, f'//*[@id="dt_purchase_digitmatch_button"]/div[1]').click()
                        print(f'====> Compra para igual: {last}')
                        sleep(2.5)
                        break
                    except: print('Erro em compra')
            else:
                navegador.find_element(By.XPATH, f'//span[contains(@class,"number") and contains(text(), {last})]').click()
        else: pass
        sleep(.1)
    except IndexError: print('Erro')

Dígito(s) com menor porcentagem: [2, 6]
Dígito(s) com menor porcentagem: [6]
Dígito(s) com menor porcentagem: [6]
====> Compra para igual: 6
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [0, 6]
Dígito(s) com menor porcentagem: [6]
Dígito(s) com menor porcentagem: [6]
====> Compra para igual: 6
Dígito(s) com menor porcentagem: [6, 9]
Dígito(s) com menor porcentagem: [6, 9]
Dígito(s) com menor porcentagem: [9]
Dígito(s) com menor porcentagem: [9]
====

In [145]:
def interceptor(request, response):
    if request.url == sk.url:
        message = {
            "proposal": 1,
            "subscribe": 1,
            "amount": 1,
            "basis": "stake",
            "contract_type": "DIGITMATCH",
            "currency": "USD",
            "symbol": "1HZ100V",
            "duration": 1,
            "duration_unit": "t",
            "barrier": 7,
            "req_id": 83
        }
        message_json = json.dumps(message)
        request.headers['Content-Type'] = 'application/json'  # Define o cabeçalho Content-Type como application/json
        request.body = message_json  # Define o corpo da solicitação como a string JSON

navegador.response_interceptor = interceptor

In [149]:
for ws in sk.ws_messages[-5:]:
    print(ws,'\n')

{"proposal":1,"subscribe":1,"amount":1,"basis":"stake","contract_type":"DIGITMATCH","currency":"USD","symbol":"1HZ100V","duration":1,"duration_unit":"t","barrier":2,"req_id":92} 

{"proposal":1,"subscribe":1,"amount":1,"basis":"stake","contract_type":"DIGITDIFF","currency":"USD","symbol":"1HZ100V","duration":1,"duration_unit":"t","barrier":2,"req_id":93} 

{"echo_req":{"amount":1,"barrier":"2","basis":"stake","contract_type":"DIGITMATCH","currency":"USD","duration":1,"duration_unit":"t","product_type":"basic","proposal":1,"req_id":92,"subscribe":1,"symbol":"1HZ100V"},"msg_type":"proposal","proposal":{"ask_price":1,"contract_details":{"barrier":"2"},"date_expiry":1688828345,"date_start":1688828344,"display_value":"1.00","id":"376e90a3-7f30-79aa-5588-3b8fd6d4e006","longcode":"Win payout if the last digit of Volatility 100 (1s) Index is 2 after 1 tick.","payout":9.09,"spot":1707.9,"spot_time":1688828344},"req_id":92,"subscription":{"id":"376e90a3-7f30-79aa-5588-3b8fd6d4e006"}} 

{"echo_re

In [33]:
lista = []


In [8]:

def contabilizador(lista_valores, limite, valor):
    lista_valores.append(int(str(valor)[-1]))
    if len(lista_valores) > limite: lista_valores.pop(0)
    valores_unicos = set(lista_valores)
    print([lista_valores.count(b)/len(lista_valores) for b in valores_unicos])
    print(lista)

def contabilizador2(lista_valores, limite, valor):
    lista_valores.append(int(str(valor)[-1]))
    if len(lista_valores) > limite:
        lista_valores.pop(0)
    
    valores_unicos = range(10)
    porcentagens = [lista_valores.count(digit) / len(lista_valores) for digit in valores_unicos]
    
    menor_porcentagem = min(porcentagens)
    menor_digit = [digit for digit, porc in zip(valores_unicos, porcentagens) if porc == menor_porcentagem]
    
    print(f"Dígito(s) com menor porcentagem: {menor_digit}")
    print(f"Lista atual: {lista_valores}")

def contabilizador3(lista_valores):
    valores_unicos = range(10)
    porcentagens = [lista_valores.count(digit) / len(lista_valores) for digit in valores_unicos]
    
    menor_porcentagem = min(porcentagens)
    menor_digit = [digit for digit, porc in zip(valores_unicos, porcentagens) if porc == menor_porcentagem]
    
    print(f"Dígito(s) com menor porcentagem: {menor_digit}")
    #print(f"Lista atual: {lista_valores}")
    if len(menor_digit) == 1:
        return (True, menor_digit[0])
    else:
        return (False, False)

contabilizador3([0,1,2,3,4,6,7,8,9])
    

Dígito(s) com menor porcentagem: [5]


(True, 5)

In [ ]:
for ws in sk.ws_messages[-50:]: 
    try: 
        valor = str(json.loads(str(ws))['tick']['quote'])
        print(0 if len(valor.split('.')[-1]) == 1 else int(str(json.loads(str(ws))['tick']['quote'])[-1]))
    except: pass

In [33]:
last_tick = [int(str(valor)[-1]) if len(str(valor).split('.')[-1]) == 2 else 0]

In [26]:
def contabilizador3(lista_valores):
    valores_unicos = range(10)
    porcentagens = [lista_valores.count(digit) / len(lista_valores) for digit in valores_unicos]
    
    menor_porcentagem = min(porcentagens)
    menor_digit = [digit for digit, porc in zip(valores_unicos, porcentagens) if porc == menor_porcentagem]
    
    print(f"Dígito(s) com menor porcentagem: {menor_digit}")
    #print(f"Lista atual: {lista_valores}")
    if len(menor_digit) == 1:
        return (True, menor_digit[0])
    else:
        return (False, False)
    
last_track = -1
meu_alvo = -1
conf_compra = 0
while True:
    try:
        if sk.ws_messages[-1:] != last_track:
            lista_last_d_tick = []
            last_track = sk.ws_messages[-1:]
            for ws in sk.ws_messages[-200:]:
                try:
                    valor = json.loads(str(ws))['tick']['quote']
                    last_tick = int(str(valor)[-1]) if len(str(valor).split('.')[-1]) == 2 else 0
                    lista_last_d_tick.append(last_tick)
                except: pass
            action, last = contabilizador3(lista_last_d_tick)
            if action == True:
                #navegador.execute_script('fetch')
                #navegador.find_element(By.XPATH, f' //span[contains(@class,"number") and contains(text(), {last})]').click() #f"//span[contains(@class,'number') and contains(text(), {last})]"
                if meu_alvo != last:
                    navegador.find_element(By.XPATH, f'//span[contains(@class,"number") and contains(text(), {last})]').click()
                    sleep(.4)
                    meu_alvo = last
                if meu_alvo == last:
                    conf_compra += 1
                if conf_compra > 2 and meu_alvo == last:
                    meu_alvo = -1
                    conf_compra = 0
                    while True:
                        try: 
                            navegador.find_element(By.XPATH, f'//*[@id="dt_purchase_digitmatch_button"]/div[1]').click()
                            print(f'==> Compra para igual: {last}')
                            sleep(.4)
                            break
                        except: print('t')
            sleep(.4)
        else: pass
    except IndexError: print('Erro')

Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1]
==> Compra para igual: 1
Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1]
==> Compra para igual: 1
Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1, 3]
Dígito(s) com menor porcentagem: [1, 3]
Dígito(s) com menor porcentagem: [1, 3]
Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1]
==> Compra para igual: 1
Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1]
t
==> Compra para igual: 1
Dígito(s) com menor porcentagem: [1]
Dígito(s) com menor porcentagem: [1, 6]
Dígito(s) com menor porcentagem: [1, 6]
Dígito(s) com menor porcentagem: [1, 4, 6]
Dígito(s) com menor porcentagem: [4]
Dígito(s) com menor porcentagem: [6]
==> Compra para igual: 6
Dígito(s) com menor porcentagem: [6]
Dígito(s) com menor porcentagem: [6]
D

KeyboardInterrupt: 

In [5]:
lista

[]

In [111]:
websocket_logs = navegador.get_log('ws_messages')

# Exibir as mensagens
for log in websocket_logs:
    print(log['message'])

InvalidArgumentException: Message: invalid argument: log type 'ws_messages' not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00606E73+48323]
	(No symbol) [0x00599661]
	(No symbol) [0x004A5308]
	(No symbol) [0x004F5B01]
	(No symbol) [0x004EA8A3]
	(No symbol) [0x004C847C]
	(No symbol) [0x004C957D]
	GetHandleVerifier [0x0086FD5D+2575277]
	GetHandleVerifier [0x008AF86E+2836158]
	GetHandleVerifier [0x008A96DC+2811180]
	GetHandleVerifier [0x006941B0+626688]
	(No symbol) [0x005A314C]
	(No symbol) [0x0059F4B8]
	(No symbol) [0x0059F59B]
	(No symbol) [0x005921B7]
	BaseThreadInitThunk [0x76317D59+25]
	RtlInitializeExceptionChain [0x7742B74B+107]
	RtlClearBits [0x7742B6CF+191]


In [62]:
def WebSocketLog():
    for wsData in navegador.get_log('performance'):
        #print(wsData) 
        wsJson = json.loads((wsData['message']))
        if wsJson["message"]["method"]== "Network.webSocketFrameReceived":
            print ("Rx :"+ str(wsJson["message"]["params"]["timestamp"]) + wsJson["message"]["params"]["response"]["payloadData"])
        if wsJson["message"]["method"] =="Network.webSocketFrameSent":
            print ("Tx :"+ wsJson["message"]["params"]["response"]["payloadData"])
WebSocketLog()

InvalidArgumentException: Message: invalid argument: log type 'performance' not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00606E73+48323]
	(No symbol) [0x00599661]
	(No symbol) [0x004A5308]
	(No symbol) [0x004F5B01]
	(No symbol) [0x004EA8A3]
	(No symbol) [0x004C847C]
	(No symbol) [0x004C957D]
	GetHandleVerifier [0x0086FD5D+2575277]
	GetHandleVerifier [0x008AF86E+2836158]
	GetHandleVerifier [0x008A96DC+2811180]
	GetHandleVerifier [0x006941B0+626688]
	(No symbol) [0x005A314C]
	(No symbol) [0x0059F4B8]
	(No symbol) [0x0059F59B]
	(No symbol) [0x005921B7]
	BaseThreadInitThunk [0x76317D59+25]
	RtlInitializeExceptionChain [0x7742B74B+107]
	RtlClearBits [0x7742B6CF+191]


In [7]:
import requests
from seleniumwire import webdriver

# Capturar o cookie do Selenium Wire
cookie = sk.response.headers.get('cookie')

# Construir a URL do WebSocket com o cookie capturado
websocket_url = sk.url

# Fazer uma solicitação GET ao WebSocket usando o cookie
response = requests.get(websocket_url, headers={'Cookie': cookie}, stream=True)

# Verificar se a solicitação foi bem-sucedida
if response.status_code == 101:
    # Iterar sobre as mensagens do WebSocket
    for message in response.iter_content(chunk_size=None):
        print(message)  # Exibir a mensagem recebida
else:
    print('Erro ao estabelecer a conexão WebSocket')

# Encerrar o driver do Selenium Wire


InvalidSchema: No connection adapters were found for 'wss://blue.binaryws.com/websockets/v3?app_id=16929&l=EN&brand=deriv'

Response(status_code=101, reason='Switching Protocols', headers=[('Date', 'Sat, 08 Jul 2023 12:26:40 GMT'), ('Content-Length', '0'), ('Connection', 'upgrade'), ('Sec-WebSocket-Accept', 'WoLjbFQTqbtvKeP7/SpDFXK+CTA='), ('Sec-WebSocket-Extensions', 'permessage-deflate'), ('Upgrade', 'websocket'), ('Content-Language', 'en'), ('CF-Cache-Status', 'DYNAMIC'), ('Server', 'cloudflare'), ('CF-RAY', '7e3851218c7502fd-GRU'), ('alt-svc', 'h3=":443"; ma=86400')], body=b'')

In [66]:
navegador.last_request

Request(method='POST', url='wss://capture.trackjs.com/capture?token=346262e7ffef497d85874322fff3bbf8&v=3.10.1', headers=[('sec-ch-ua', '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"'), ('sec-ch-ua-platform', '"Windows"'), ('sec-ch-ua-mobile', '?0'), ('content-type', 'text/plain'), ('accept', '*/*'), ('sec-fetch-site', 'cross-site'), ('sec-fetch-mode', 'cors'), ('sec-fetch-dest', 'empty'), ('referer', 'https://app.deriv.com/'), ('accept-encoding', 'gzip, deflate, br'), ('content-length', '0'), ('user-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'), ('origin', 'https://app.deriv.com'), ('accept-language', 'en-US,en;q=0.9'), ('Host', 'blue.binaryws.com'), ('Connection', 'Upgrade'), ('Upgrade', 'websocket'), ('Sec-WebSocket-Version', '13'), ('Sec-WebSocket-Key', 'bZtx66I63GsywoL6BVz7tw=='), ('Sec-WebSocket-Extensions', 'permessage-deflate; client_max_window_bits'), ('Pragma', 'no-cache'), ('Cache-